## Datasource
- [x] `classification_dataset` from Jung+Ninja
- [x] `Another Website` include: amarintv, , ch3thailand, instagram, mgronline,pantip, pptv, sanook, sudsapda, thairath,       tnn2, workpointnews, babyandkids, dailynews	, khaosod,  new18, postjung, praew, siamdara,thaich8,thansettakij, tvpoolonline, brighttv, facebook, matichon, one31, posttoday, ryt9, siamsport, thainews_prd, think_of_living, twitter
- [ ] `social_listening` include: Facebook, Twitter and Instagram
- [x] `Wiki TH`
- [x] `Pantip` of Knight (data_lm)

## Stucture Files
```
/raw_data_extraction
│   README.md
└───classification_dataset
│   └───scrapy_dailynews
│   |   │   file111.txt
│   └───scrapy_naewna
│   
└───Another Website
│   └───amarintv
│   |   │   file111.txt
│   └───baanlaesuan
│       │   file111.txt
│   
└───social_listening 
│   │   comment_data.txt
│   │   pantip_post_data.txt
│   
└───Wiki TH
│   └───AA
│       │   file111.txt
│   
└───Pantip 
│   │   file011.txt
```

### Classification_dataset from Jung + Ninja
```
print(data['title'])
print(data['author'])
print(data['publish'])
print(data['content_id'])
print(data['content'])
print(data['url'])
print(data['images'])
print(data['content_class'])
```

In [1]:
# Clear Files
# !rm -rf /workdir/Code/bma_transformer_model/data/raw_data_extraction/*
# --------

In [2]:
import json
import os
import pandas as pd
import re
from enum import Enum
import os.path

In [3]:
from functools import reduce

def normalize_unicode_and_lowercase(content):
    from tokenizers import Tokenizer, models
    from tokenizers.normalizers import Lowercase, NFKC, Sequence
    tokenizer = Tokenizer(models.BPE())
    tokenizer.normalizer = Sequence([
        NFKC(),
        Lowercase()
    ])
    return tokenizer.normalize(content)
def filter_wiki_extracted_text(content):
    return re.sub('<doc.*>.*[\n\r]+|<\/doc>.*[\n\r]+', '', content)

def filter_all_double_enters(content):
    return re.sub('[\n\r]{2,}', '\n', content)

# Thai use double spaces for a reason, so [ ]{3, } is used instead of {2, }
#  Also filter the last enter with \n$
def filter_all_double_spaces_tabs(content):
    return re.sub('[\t]|[ ]{3,}|\n$', '', content)



def write_file_from_text(filename, data, TYPE_DATA, idx=0):
    f=open(filename, 'a')
    
    normalization_functions = [normalize_unicode_and_lowercase, filter_wiki_extracted_text, 
                              filter_all_double_spaces_tabs,  filter_all_double_enters, ]
    
    # For p'Moo ; jung ninja Classification ; thwiki
    if (TYPE_DATA == "TEXT" or TYPE_DATA == "WIKI"):
        data_buffer = reduce(lambda res, f: f(res), normalization_functions, data )

    # for Pantip Data and social listening
    elif (TYPE_DATA == "dataLM"):
        data_buffer = reduce(lambda res, f: f(res), normalization_functions, data['Title'][idx] )
        data_buffer += "\n"
        data_buffer += reduce(lambda res, f: f(res), normalization_functions,  data['Kratoo'][idx])
    elif (TYPE_DATA == "social_listening"):
        # print("?>>?", data[idx])
        try:
            # print("?>>?", data[idx])
            if (data[idx] != None):
                data_buffer = reduce(lambda res, f: f(res), normalization_functions, data[idx] )
        except:
            print("ERROR")
    
    # Write the new Data
    try:
        f.write(data_buffer)
        # New Content
        f.write('\n\n')
    except:
        print("ERROR")
    return f.tell()

def write_file_from_json(filename, data, idx=0):
    f=open(filename,'a')
    
    normalization_functions = [normalize_unicode_and_lowercase, filter_wiki_extracted_text, 
                              filter_all_double_spaces_tabs,  filter_all_double_enters, ]
    data_buffer = reduce(lambda res, f: f(res), normalization_functions, data['title'] )
    data_buffer += "\n"
    data_buffer += reduce(lambda res, f: f(res), normalization_functions, data['content'])
    
    # Write the new Data
    f.write(data_buffer)
    # New Content
    f.write('\n\n')
    return f.tell()


In [4]:
ROOT_DIR = "/workdir/Code/bma_transformer_model/data/raw/"
EXPORT_DIR = "/workdir/Code/bma_transformer_model/data/raw_data_extraction/"

# SELECTOR_DIR = ["classification_dataset", "another_website", "thwiki-20200601-extracted", "data_lm", "social_listening"]
# SELECTOR_DIR = ["another_website"]
# SELECTOR_DIR = ["classification_dataset"]
# SELECTOR_DIR = ["thwiki-20200601-extracted"]
# SELECTOR_DIR = ["data_lm"]
SELECTOR_DIR = ["social_listening"]

MINIMUM_FILE_SIZE = 30000000 #size of file 30 MB 
OUTPUT_Filename = ""
extension = ".txt"
TEMP_Filename = "temp_data.txt"
count = 1


def create_folder(pathFile, count):
    if not os.path.exists(os.path.dirname(pathFile)):
        try:
            os.makedirs(os.path.dirname(pathFile))
            count = 0
            return count
        except OSError as exc: # Guard against race condition
            raise
    return count


def readAndWriteFromJSONFile_LimitFileSize(pathFile, output_file, count):
    # Read JSON File
    with open(pathFile) as json_file:
        data = json.load(json_file)
        
        # Write TxT File
        newSize = write_file_from_json(output_file + "_" + str(count) + ".txt", data)
        # IF size of file MINIMUM_FILE_SIZE 
        if(newSize > MINIMUM_FILE_SIZE):
            count += 1
    return count

def readAndWriteFromTEXTFile_LimitFileSize(pathFile, output_file, count, TYPE_DATA):
    try:
        fRead = open(pathFile, "r")
        data = fRead.read()
    except:
        data = ""

    # Write TxT File on Temp File
    newSize = write_file_from_text(output_file + "_" + str(count) + '.txt', data, TYPE_DATA)

    # IF size of file MINIMUM_FILE_SIZE
    if(newSize > MINIMUM_FILE_SIZE):
        count += 1
    return count


def readAndWriteCsvFile_LimitFileSize(pathFile, output_file, count, dataFrame, TYPE_DATA):
    for i in range(0, dataFrame.shape[0]):
        newSize = write_file_from_text(output_file + "_" + str(count) + '.txt', dataFrame, TYPE_DATA, idx=i)

        # IF size of file MINIMUM_FILE_SIZE
        if(newSize > MINIMUM_FILE_SIZE):
            count += 1
    return count
    

In [5]:
for selector_dir in SELECTOR_DIR:
    old_folder_file = ""
    temp_filename = ""
    for root, dirs, files in os.walk(os.path.join(ROOT_DIR, selector_dir)):
        for file in files:
            folder = os.path.basename(root)

            # Type: "classification_dataset"
            if (selector_dir == "classification_dataset" and folder != "full"):
                # Input
                pathFile = os.path.join(root, file)
                filename, file_extension = os.path.splitext(pathFile)
                folder_file = os.path.basename(root)

                # Output
                output_file = os.path.join(EXPORT_DIR, selector_dir, folder_file)
                if(file_extension != '.jpg' or file_extension != '.xml' or file_extension != '.py'):
                    count = create_folder(output_file, count)
                    
                    if(old_folder_file != folder_file):
                        count = 0
                        old_folder_file = folder_file
                        
                    #Read & Write New File
                    count = readAndWriteFromJSONFile_LimitFileSize(pathFile, output_file, count)
                    
            elif (selector_dir == "another_website"):
                # Input
                pathFile = os.path.join(root, file)
                folder_file = pathFile.split("/")[-3]

                # Output
                output_file = os.path.join(EXPORT_DIR, selector_dir, folder_file)

                if(old_folder_file != folder_file):
                    count = 0
                    old_folder_file = folder_file

                count = create_folder(output_file, count)
                count = readAndWriteFromTEXTFile_LimitFileSize(pathFile, output_file, count, "TEXT")

            elif (selector_dir == "thwiki-20200601-extracted"):
                # Input
                pathFile = os.path.join(root, file)
                folder_file = pathFile.split("/")[-2]

                # Output
                output_file = os.path.join(EXPORT_DIR, selector_dir, "Wiki"+folder_file)
                
                if(old_folder_file != folder_file):
                    count = 0
                    old_folder_file = folder_file

                count = create_folder(output_file, count)
                count = readAndWriteFromTEXTFile_LimitFileSize(pathFile, output_file, count, "WIKI")
            elif (selector_dir == "data_lm"):
                # Input
                pathFile = os.path.join(root, file)
                folder_file = pathFile.split("/")[-1]

                # Output
                output_file = os.path.join(EXPORT_DIR, selector_dir, "Pantip"+folder_file)
                
                if(old_folder_file != folder_file):
                    count = 0
                    old_folder_file = folder_file
                count = create_folder(output_file, count)
                dataFrame = pd.read_csv(pathFile, delimiter=',', encoding='utf-8')
                count = readAndWriteCsvFile_LimitFileSize(pathFile, output_file, count, dataFrame, "dataLM")
            elif (selector_dir == "social_listening"):
                pathFile = os.path.join(root, file)
                folder_file = pathFile.split("/")[-1]

                # Output
                output_file = os.path.join(EXPORT_DIR, selector_dir, "SocialListening"+folder_file)
                
                if(old_folder_file != folder_file):
                    count = 0
                    old_folder_file = folder_file
                count = create_folder(output_file, count)
                pathFile = "/workdir/Code/bma_transformer_model/data/raw/social_listening/pantip_post_data.csv"
                dataFrame = pd.read_csv(pathFile, encoding='utf-8', delimiter="\t", error_bad_lines=False, header=None, engine='python')
                data_input = []
                print("file", file)
                if(file == "post_data.csv" or file == "pantip_post_data.csv"):
                    print(dataFrame[4])
                    data_input = dataFrame[4]
                elif(file == "comment_data.csv"):
                    data_input = dataFrame[8]
                count = readAndWriteCsvFile_LimitFileSize(pathFile, output_file, count, data_input, "social_listening")

                

Skipping line 750: '	' expected after '"'
Skipping line 1035: '	' expected after '"'
Skipping line 1036: '	' expected after '"'
Skipping line 1389: '	' expected after '"'
Skipping line 2625: '	' expected after '"'
Skipping line 2637: '	' expected after '"'
Skipping line 2920: '	' expected after '"'
Skipping line 2961: '	' expected after '"'
Skipping line 2965: '	' expected after '"'
Skipping line 2973: '	' expected after '"'
Skipping line 3002: '	' expected after '"'
Skipping line 3303: '	' expected after '"'
Skipping line 4517: '	' expected after '"'
Skipping line 4604: '	' expected after '"'
Skipping line 5201: '	' expected after '"'
Skipping line 5350: '	' expected after '"'
Skipping line 6690: '	' expected after '"'
Skipping line 7434: '	' expected after '"'
Skipping line 8800: '	' expected after '"'
Skipping line 8871: '	' expected after '"'
Skipping line 9013: '	' expected after '"'
Skipping line 9227: '	' expected after '"'
Skipping line 9664: '	' expected after '"'
Skipping lin

Skipping line 100778: '	' expected after '"'
Skipping line 102591: '	' expected after '"'
Skipping line 102592: '	' expected after '"'
Skipping line 103081: '	' expected after '"'
Skipping line 104276: '	' expected after '"'
Skipping line 104596: '	' expected after '"'
Skipping line 104901: '	' expected after '"'
Skipping line 105381: '	' expected after '"'
Skipping line 105811: '	' expected after '"'
Skipping line 106184: '	' expected after '"'
Skipping line 106185: '	' expected after '"'
Skipping line 106861: '	' expected after '"'
Skipping line 107669: '	' expected after '"'
Skipping line 108014: '	' expected after '"'
Skipping line 109114: '	' expected after '"'
Skipping line 109134: '	' expected after '"'
Skipping line 109451: '	' expected after '"'
Skipping line 110300: '	' expected after '"'
Skipping line 110872: '	' expected after '"'
Skipping line 110936: '	' expected after '"'
Skipping line 111379: '	' expected after '"'
Skipping line 111584: '	' expected after '"'
Skipping l

Skipping line 235535: '	' expected after '"'
Skipping line 235733: '	' expected after '"'
Skipping line 236057: '	' expected after '"'
Skipping line 236912: '	' expected after '"'
Skipping line 237566: '	' expected after '"'
Skipping line 237567: '	' expected after '"'
Skipping line 237913: '	' expected after '"'
Skipping line 239930: '	' expected after '"'
Skipping line 240167: '	' expected after '"'
Skipping line 240734: '	' expected after '"'
Skipping line 240828: '	' expected after '"'
Skipping line 241225: '	' expected after '"'
Skipping line 241742: '	' expected after '"'
Skipping line 242437: '	' expected after '"'
Skipping line 242726: '	' expected after '"'
Skipping line 243419: '	' expected after '"'
Skipping line 244286: '	' expected after '"'
Skipping line 244449: '	' expected after '"'
Skipping line 245195: '	' expected after '"'
Skipping line 245887: '	' expected after '"'
Skipping line 246471: '	' expected after '"'
Skipping line 246962: '	' expected after '"'
Skipping l

Skipping line 327420: '	' expected after '"'
Skipping line 327474: '	' expected after '"'
Skipping line 328704: '	' expected after '"'
Skipping line 328950: '	' expected after '"'
Skipping line 330825: '	' expected after '"'
Skipping line 330832: '	' expected after '"'
Skipping line 330851: '	' expected after '"'
Skipping line 331438: '	' expected after '"'
Skipping line 331567: '	' expected after '"'
Skipping line 331824: '	' expected after '"'
Skipping line 332204: '	' expected after '"'
Skipping line 332208: '	' expected after '"'
Skipping line 332370: '	' expected after '"'
Skipping line 332548: '	' expected after '"'
Skipping line 332552: '	' expected after '"'
Skipping line 333137: '	' expected after '"'
Skipping line 334523: '	' expected after '"'
Skipping line 335244: '	' expected after '"'
Skipping line 335409: '	' expected after '"'
Skipping line 338002: '	' expected after '"'
Skipping line 338213: '	' expected after '"'
Skipping line 338299: '	' expected after '"'
Skipping l

Skipping line 460069: '	' expected after '"'
Skipping line 461272: '	' expected after '"'
Skipping line 461530: '	' expected after '"'
Skipping line 462658: '	' expected after '"'
Skipping line 462909: '	' expected after '"'
Skipping line 464520: '	' expected after '"'
Skipping line 465976: '	' expected after '"'
Skipping line 466631: '	' expected after '"'
Skipping line 467439: '	' expected after '"'
Skipping line 467440: '	' expected after '"'
Skipping line 467455: '	' expected after '"'
Skipping line 469865: '	' expected after '"'
Skipping line 470724: '	' expected after '"'
Skipping line 471069: '	' expected after '"'
Skipping line 471080: '	' expected after '"'
Skipping line 471552: '	' expected after '"'
Skipping line 472539: '	' expected after '"'
Skipping line 472702: '	' expected after '"'
Skipping line 472844: '	' expected after '"'
Skipping line 472847: '	' expected after '"'
Skipping line 472866: '	' expected after '"'
Skipping line 473330: '	' expected after '"'
Skipping l

...

Skipping line 1079266: '	' expected after '"'
Skipping line 1079507: '	' expected after '"'
Skipping line 1079942: '	' expected after '"'
Skipping line 1080381: '	' expected after '"'
Skipping line 1080576: '	' expected after '"'
Skipping line 1081262: '	' expected after '"'
Skipping line 1081612: '	' expected after '"'
Skipping line 1081626: '	' expected after '"'
Skipping line 1081879: '	' expected after '"'
Skipping line 1082377: '	' expected after '"'
Skipping line 1082408: '	' expected after '"'
Skipping line 1082983: '	' expected after '"'
Skipping line 1083868: '	' expected after '"'
Skipping line 1084325: '	' expected after '"'
Skipping line 1084338: '	' expected after '"'
Skipping line 1085099: '	' expected after '"'
Skipping line 1085237: '	' expected after '"'
Skipping line 1085989: '	' expected after '"'
Skipping line 1086839: '	' expected after '"'
Skipping line 1086933: '	' expected after '"'
Skipping line 1086935: '	' expected after '"'
Skipping line 1087210: '	' expecte

Skipping line 1189620: '	' expected after '"'
Skipping line 1190367: '	' expected after '"'
Skipping line 1190683: '	' expected after '"'
Skipping line 1191396: '	' expected after '"'
Skipping line 1191716: '	' expected after '"'
Skipping line 1191884: '	' expected after '"'
Skipping line 1192677: '	' expected after '"'
Skipping line 1192718: '	' expected after '"'
Skipping line 1192722: '	' expected after '"'
Skipping line 1192727: '	' expected after '"'
Skipping line 1192790: '	' expected after '"'
Skipping line 1194955: '	' expected after '"'
Skipping line 1194974: '	' expected after '"'
Skipping line 1194975: '	' expected after '"'
Skipping line 1194980: '	' expected after '"'
Skipping line 1196889: '	' expected after '"'
Skipping line 1197839: '	' expected after '"'
Skipping line 1198105: '	' expected after '"'
Skipping line 1198199: '	' expected after '"'
Skipping line 1198278: '	' expected after '"'
Skipping line 1198623: '	' expected after '"'
Skipping line 1198838: '	' expecte

Skipping line 1302049: '	' expected after '"'
Skipping line 1302169: '	' expected after '"'
Skipping line 1302796: '	' expected after '"'
Skipping line 1303628: '	' expected after '"'
Skipping line 1304582: '	' expected after '"'
Skipping line 1305491: '	' expected after '"'
Skipping line 1307121: '	' expected after '"'
Skipping line 1307182: '	' expected after '"'
Skipping line 1307184: '	' expected after '"'
Skipping line 1307354: '	' expected after '"'
Skipping line 1307369: '	' expected after '"'
Skipping line 1307557: '	' expected after '"'
Skipping line 1308650: '	' expected after '"'
Skipping line 1308729: '	' expected after '"'
Skipping line 1309130: '	' expected after '"'
Skipping line 1309842: '	' expected after '"'
Skipping line 1309952: '	' expected after '"'
Skipping line 1310407: '	' expected after '"'
Skipping line 1310735: '	' expected after '"'
Skipping line 1310978: '	' expected after '"'
Skipping line 1311479: '	' expected after '"'
Skipping line 1314068: '	' expecte

file page_data.csv


AttributeError: 'list' object has no attribute 'shape'

In [ ]:
pathFile = "/workdir/Code/bma_transformer_model/data/raw/social_listening/pantip_post_data.csv"
df = pd.read_csv(pathFile, encoding='utf-8', delimiter="\t", error_bad_lines=False, header=None)
    

In [ ]:
print(df[4])

In [ ]:
# pathFile = "/workdir/Code/bma_transformer_model/data/raw/social_listening/post_data.csv"
pathFile = "/workdir/Code/bma_transformer_model/data/raw/social_listening/comment_data.csv"
df = pd.read_csv(pathFile, encoding='utf-8', delimiter="\t", error_bad_lines=False, header=None, engine='python')
    

In [ ]:
df[8]

# Testing Area

In [ ]:
wiki_extracted_text = """<doc id="1" url="https://th.wikipedia.org/wiki?curid=1" title="หน้าหลัก">
หน้าหลัก


วิกิพีเดียดำเนินการโดยมูลนิธิวิกิมีเดีย องค์กรไม่แสวงผลกำไร ผู้ดำเนินการอีกหลาย ได้แก่

 __NOEDITSECTION__

</doc>
<doc id="545" url="https://th.wikipedia.org/wiki?curid=545" title="ดาราศาสตร์">
ดาราศาสตร์

ดาราศาสตร์ คือวิชาวิทยาศาสตร์ที่ศึกษาวัตถุท้องฟ้า (อาทิ ดาวฤกษ์ ดาวเคราะห์ ดาวหาง และดาราจักร) รวมทั้งปรากฏการณ์ทางธรรมชาติต่าง ๆ ที่เกิดขึ้นจากนอกชั้นบรรยากาศของโลก โดยศึกษาเกี่ยวกับวิวัฒนาการ ลักษณะทางกายภาพ ทางเคมี ทางอุตุนิยมวิทยา และการเคลื่อนที่ของวัตถุท้องฟ้า ตลอดจนถึงการกำเนิดและวิวัฒนาการของเอกภพ"""


In [ ]:
many_space_text= """test1


3 enters



4 enters
1 enter
hello i like   spaces
    Tabbing too?
    Another paragraph
lalalala
\tReal Tabs
\t\t Real tTasdoij12

2 enters

"""

In [ ]:
daily_news_text="""รายงานสภาพจราจรเมืองกรุงและพื้นที่ใกล้เคียงชั่วโมงเร่งด่วน ช่วงเย็นวันที่ 16 เม.ย. 63 เพื่อเป็นข้อมูลสำหรับผู้ใช้รถใช้ถนนในการสัญจรไปยังสถานที่ต่างๆ                    \r\n                    \r\n                    17.30 น. ถนนวิภาวดีรังสิตขาเข้า มุ่งหน้า ม.เกษตรฯ รถน้อยเคลื่อนตัวได้ฉลุย ฝั่งตรงข้ามขาออกถนนโล่งเคลื่อนตัวได้ดี\n\r\n16.54 น. รถจอดเสีย ถนนเเจ้งวัฒนะ ขาเข้า ปากซอยเเจ้งวัฒนะ 6 รถบรรทุกทราย จอดเสียเลนซ้าย ประสานงานเจ้าหน้าที่ทราบแล้ว\n\r\n16:25 น. #มอเตอร์เวย์ ช่วง ด่านเก็บเงินลาดกระบัง > ด่านเก็บเงินบางบ่อ  ที่กม. 36+600 มีรถปิคอัพพลิกตะแคง ขวาง 2 ช่องทางขวา การจราจรเคลื่อนตัวได้\n\r\n16.00 น. อุบัติเหตุ ถนนสายไหม ขาเข้า ปากซอยสายไหม 56 รถชนกัน 3 คัน (เก๋ง 2 / สิบล้อ 1) กีดขวางเลนกลาง การจราจรติดขัด\n\r\nขอบคุณ @longdotraffic @Traffic_119, @js100radio, @fm91trafficpro, 1197สายด่วนจราจร,และผู้ใช้ทวิตเตอร์"""

In [ ]:
# For Testing
# print(filter_all_double_spaces_tabs(filter_all_double_enters(many_space_text)))
# funcs = [normalize_unicode_and_lowercase, filter_wiki_extracted_text, filter_all_double_enters, filter_all_double_spaces_tabs] 
# print(reduce(lambda res, f: f(res), funcs, wiki_extracted_text ))

In [ ]:
normalization_functions = [normalize_unicode_and_lowercase, filter_wiki_extracted_text, 
                              filter_all_double_spaces_tabs,  filter_all_double_enters, ]

In [ ]:
print(daily_news_text)

In [ ]:
print(reduce(lambda res, f: f(res), normalization_functions, daily_news_text ))